  <picture>
  <source srcset="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes-w.svg" media="(prefers-color-scheme: dark)">
    <img src="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes.svg">
  </picture>

Rosette101
===

The previous post was about getting familiar with Racket. To make things more interesting we have used Jupyter notebook interactive environment. This time was are going to use Rosette in the same environment.

Rosette is a solver-aided programming language. To be specific popular [Z3](https://github.com/Z3Prover/z3) is behind the curtains of Rosette. With the power of Z3, Rosette programs can understand and handle symbolic values, assertions, assumptions, and queries. Which is a vital part of verifying firmware and hardware designs. Lets go through example use cases provided in the [Rosette documentation](https://docs.racket-lang.org/rosette-guide/index.html). 

> Note: Rosette's symbolic evaluation engine operates on the program states. Instead of operating on one path at a time, it keeps track of all assumptions and assertions in the path at the beginning.  This is done through the `verification` condition (VC)`.  During the tutorial, we will be switching contexts between different examples and problems. Since we are using the same kernel to run different examples, we need to clean the VC when we switch the context.

First we start with the language definition,

In [1]:
#lang iracket/lang #:require rosette/safe 

Symbolic Values
---

When it comes to validating designs and implementations, one thing is we can't visualize all the internal variables. These variables are sometimes constrained by different conditions. With Rosette, we can keep these variables as unknown variable variables which refers to the technical term of ```symbolic variables```. These variables are symbolically evaluated on the constraints that act on them. 

In [2]:
(define-symbolic b boolean?)
(displayln b)

b


In [3]:
(displayln (boolean? b))

#t


In [4]:
(displayln (integer? b))

#f


In [5]:
(displayln (vector b 1))

#(b 1)


In [6]:
(displayln (not b))

(! b)


(displayln (boolean? (not b)))

Assertions and Assumptions
---

As the name suggests, Rosette's assertions work exactly similarly to assertions used in other languages. If the assertion is failed, it will terminate the exection with an exception.

In [10]:
(assert #t)
(assert #f)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level


Since the assertion is failed, this wiil be consistant within the verification context. Therefore, we need to clear the verification conditions if they are no longer valid for the verifcation scenario under consideration.

In [11]:
(clear-vc!)

In [12]:
(define-symbolic i j integer?)
(assume (> j 0))     ; Add the assumption (> j 0) to the VC.
(vc-assumes (vc))

(< 0 j)

In [13]:
(assert (< (- i j) i))
(vc-asserts (vc))    ; The assertions must hold when the assumptions hold.

(display (vc))

#(struct:vc (< 0 j) (|| (! (< 0 j)) (< (+ i (- j)) i)))

In [15]:
(clear-vc!)

Solver-Aided Queries
---

Assertions can be performed on solver queries. As as example we define the function below utilizing two different methods. First one define the polynomial and second one define the factorized polynomial. We can assert the function for equivelence using rosette in following manner.

In [19]:
(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x 1) (+ x 2) (+ x 2)))
 
(define (same p f x)
 (assert (= (p x) (f x))))

In [20]:
(same poly factored 0)
(same poly factored -1)
(same poly factored -2)

Now let's try this with two different functions, to check whether the assetion will fail.

In [23]:
(define (factored-2 x)
 (* x (+ x 3) (+ x 2) (+ x 2)))

In [24]:
(same poly factored-2 0)
(same poly factored-2 -1)
(same poly factored-2 -2)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level
   eval:7:0: same
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


As expected assertion fails. Remember we need to clear the `vc` to avoid confusion later.

In [26]:
(clear-vc!)

Verification
---

Program verification solves the problem of verifying the program for all the legal inputs. The brute force approach would be to iterate through all the values one by one. Although this sounds promising at capturing any bug if exists, in the case of a 32-bit adder circuit, input space would be $2^{64}$ which is a large number.
Instead of going through the brute force path, Rosette delegates this task to `z3 constraint solver` with the help of `verify` query.


In [27]:
(define-symbolic i integer?)
(define cex (verify (same poly factored i)))

In [28]:
(evaluate i cex)
(poly -6)
(factored -6)
; (same poly factored -6)

480

In [29]:
(clear-vc!)


Synthesis
----

The solver can not only find failure-inducing inputs and localize faults, it can also synthesize repairs for buggy expressions. To repair a program, we first replace each buggy expression with a syntactic "hole." A program with holes is called a sketch. The solver completes a sketch by filling its holes with expressions, in such a way that all assertions in the resulting program pass on all inputs.

The following code snippet shows the sketch for our buggy factored procedure. We obtained it by replacing the constants in the minimal core with (??) holes, which are filled with numerical constants.4

In [30]:
#lang iracket/lang #:require rosette/safe

In [2]:
(require rosette/safe)
(require rosette/lib/synthax)

(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x (??)) (+ x (??)) (+ x (??))))
 
(define (same p f x)
 (assert (= (p x) (f x))))


In [3]:
(define-symbolic i integer?)
(define binding
    (synthesize #:forall (list i)
                #:guarantee (same poly factored i)))
;(print-forms binding)
(displayln binding)

(model
 [??:eval:8:12 1]
 [??:eval:8:23 2]
 [??:eval:8:34 3])


Note that we can't use `print-forms` within Jupyter Notebook since we can save the program directly to Disc. Instead, we print the model and it will show the solution for three holes (??) that we have created.

In [27]:
(require racket/include)
(include "rosette101/synthesis.rkt")


binding

open-input-file: contract violation
  expected: path-string?
  given: 'eval
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/lib/util/syntax.rkt:32:0: read-module
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/lib/synthax.rkt:243:0: body of top-level
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/lib/synthax.rkt:296:0: print-forms


In [4]:
(clear-vc!)


Angelic Execution
----

In [5]:
(define-symbolic x y integer?)
(define sol
    (solve (begin (assert (not (= x y)))
                  (assert (< (abs x) 10))
                  (assert (< (abs y) 10))
                  (assert (not (= (poly x) 0)))
                  (assert (= (poly x) (poly y))))))
(evaluate x sol)

(evaluate y sol)

(evaluate (poly x) sol)

(evaluate (poly y) sol)

24

In [6]:
(clear-vc!)

#### Lets Do an experiment
For this experiment, Let's generate two implementations of 32-bit adders which utilize two different algorithms. Our task is to verify these two implementations. 

In [3]:
#lang iracket/lang #:require rosette/safe

(require rosette)
(require rosette/safe)
(require racket)
(require racket/vector)

First adder implementation is as below,

In [4]:
(define (add-vectors a b)
  (define (full-adder x y c-in)
    (let* ((sum (bitwise-xor x y c-in))
           (carry (bitwise-ior (bitwise-and x y) (bitwise-and c-in (bitwise-xor x y)))))
      (cons sum carry)))
  (let* ((result (make-vector 32 0))
         (carry 0))
    (for ([i (in-range 31 -1 -1)])
      (let* ((full-add (full-adder (vector-ref a i) (vector-ref b i) carry))
             (sum (car full-add))
             (carry (cdr full-add)))
        (vector-set! result i sum)))
    result))

Second implementation

In [5]:
(define (add-vectors2 vec1 vec2)
  (define result (make-vector 32 0))
  (define carry 0)
  (let loop ((i 31))
    (cond
      ((< i 0) result)
      (else
       (let ((sum (+ (vector-ref vec1 i) (vector-ref vec2 i) carry)))
         (vector-set! result i (bitwise-and sum #xffffffff))
         (set! carry (if (< sum #x100000000) 0 1))
         (loop (- i 1)))))))

Test for several test cases

In [6]:
(displayln (add-vectors #(0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1)
               #(1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1)))
(displayln (add-vectors2 #(0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1)
               #(1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1)))

#(1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0)
#(1 1 1 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 2)


Apperently from the test case, two functions are not equal. Let's check for the equivelence of the two methods using formal methods.

Test with several test cases

In [7]:
(define a (make-vector 32 'symbolic-value))
(define b (make-vector 32 'symbolic-value))

; Using Rosette verify query
(define (check) 
   (set! output1 (add-vectors a b))
   (set! output2 (add-vectors2 a b))
   (verify
      (assert(equal?  output1 output2 ))))

check

; (check a b)
; (evaluate check (list a b))

#<procedure:check>

In [8]:
(verify
   (assert(equal? (add-vectors a b) (add-vectors2 a b))))

(model)

In [28]:
#lang iracket/lang #:require rosette/safe

In [29]:


(define (quadratic-roots a b c)
  (define delta (- (* b b) (* 4 a c)))
  (if (< delta 0)
      #f
      (let* ([sqrt-delta (sqrt delta)]
             [denominator (* 2 a)])
        (list (/ (- (- b) sqrt-delta) denominator)
              (/ (- (- b) (- sqrt-delta)) denominator)))))

In [30]:

(define-syntax-rule (check-equal actual expected)
  (if (equal? actual expected)
      (void)
      (error 'check-equal "expected: ~s, got: ~s" expected actual)))

(check-equal (quadratic-roots 1 -5 6) '(2 3))
(check-equal (quadratic-roots 1 2 1) '(-1 -1))
(check-equal (quadratic-roots 1 -2 1) '(1 1))
(check-equal (quadratic-roots 1 1 1) #f)

sqrt: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/form/control.rkt:16:25
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


### Some Problems from Internet


Finding an integer whose absolute value is X with Racket

In [1]:
#lang iracket/lang #:require rosette/safe 

(define (abs x)
    (if (< x 0) (- x) x))

In [2]:
; define a symbolic variable y

(define-symbolic y integer?)

In [3]:
; Solve a constraint saying |y| = 5.
(solve
  (assert (= (abs y) 5)))

(model
 [y -5])

In [31]:
(clear-vc!)


Now let us look at some sample questions from CAV 2019 which was developed by [Emina](https://github.com/emina)




#### Q1. Warmup

Suppose that you are an app developer for a super low-power chip that achieves peak energy efficiency on programs with few or no branches. To build apps faster, you hire Ben Bitdiddle to create a library of efficient primitives for this chip, starting with a branch-free function that computes the absolute value of a 32-bit bitvector. 

In [7]:
#lang iracket/lang #:require rosette

In [8]:
(define (abs-spec x)
  (if (bvslt x (bv 0 32))
      (bvneg x)
      x))

Ben comes up with the following implementation that works on his test suite:

In [9]:
(define (abs-impl x) 
  (let* ([o1 (bvashr x (bv 31 32))]
         [o2 (bvadd x o1)]
         [o3 (bvsub o2 o1)])
    o3))

; zero, positive, negative ...
(assert (equal? (abs-impl (bv #x00000000 32)) 
                  (abs-spec (bv #x00000000 32))))
(assert (equal? (abs-impl (bv #x00000003 32)) 
                  (abs-spec (bv #x00000003 32))))
(assert (equal? (abs-impl (bv #x80000000 32)) 
                  (abs-spec (bv #x80000000 32))))

Help review Ben’s code by answering the following questions. If a question requires you to modify his implementation in any way, do so on a fresh copy with a new name such as abs-impl-N where N is the question number.

Does Ben’s implementation work on all 32-bit inputs? Use the verify query to check.

In [11]:
; Let's define a generic 32bit vector
(define-symbolic x (bitvector 32))

; Using Rosette verify query
(define check
  (verify
   (assert(equal? (abs-spec x) (abs-impl x)))))

(evaluate check x)

(model
 [x (bv #x8404c7c0 32)])

Apparently, we have a counter-example where implementation does not satisfy the specification.

The verifier will reveal that Ben’s implementation is buggy. Use debug to localize the fault found by the verifier (and remember to save your work to a file before invoking debug).

Use synthesize to fix Ben’s implementation as suggested by the debugger. You can assume that if Ben has made an error, it is always of the same kind: using an arithmetic operator (such as addition or subtraction) instead of a bitwise operator that takes the same arguments. It is easiest to sketch this knowledge using the choose construct.

In [10]:
(require rosette/lib/synthax)

(define (abs-impl-3 x) 
  (let* ([o1 (bvashr x (bv 31 32))]
         [o2 ((choose bvadd bvand bvor bvxor bvshl bvlshr bvashr) x o1)]
         [o3 ((choose bvsub bvand bvor bvxor bvshl bvlshr bvashr) o2 o1)])
    o3))


In [12]:
(synthesize
  #:forall x
  #:guarantee (assert (equal? (abs-spec x) (abs-impl-3 x))))

(model
 [0$choose:eval:5:15 #t]
 [1$choose:eval:5:15 #f]
 [2$choose:eval:5:15 #f]
 [3$choose:eval:5:15 #f]
 [4$choose:eval:5:15 #f]
 [5$choose:eval:5:15 #f]
 [0$choose:eval:6:15 #f]
 [1$choose:eval:6:15 #f]
 [2$choose:eval:6:15 #f]
 [3$choose:eval:6:15 #t])

#### Q2. Majority Voting Algorithm

The [Boyer–Moore majority vote algorithm](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore_majority_vote_algorithm) is a beautiful and tricky procedure for finding a majority element in a list, using linear time and constant space.

Complete the following Rosette implementation of Boyer-Moore and verify its correctness for any list of N arbitrary integers.

In [2]:
#lang iracket/lang #:require rosette

In [32]:
(define (boyer-moore xs)
  (define m (car xs))  ; car returns the first element of an array
  (define i 0)
  (for ([x xs])
    (if (= i 0) (and (set! m x) (set! i 1))
          (if (= m x) (set! i (+ i 1)) 
          (set! i (- i 1)))
          ))
  m)

Test for several test cases in traditional way

In [3]:
(display "case 1 :")(displayln (= 1 (boyer-moore `(1 1 2 4 1 1))))
(display "case 2 :")(displayln (= 4 (boyer-moore `(8 6 4 1 2 4 1 1 8 4 8 4 4))))

case 1 :#t
case 2 :#t


In [33]:
;Check the algorithm for a list of n arbitrary integers.
(define (check n [bw #f])
  (current-bitwidth bw)
  (define-symbolic* xs integer? [n])
  (define-symbolic* m integer?)
  (verify 
  #:assume ; there is a majority value ...
   (assert (> (count (curry = m) xs)
              (quotient n 2)))
   #:guarantee ; algorithm works
   (assert (= m (boyer-moore xs)))))

(time (check 10)) 



eval:6:2: verify: use does not match pattern: (verify expr)
  in: (verify #:assume (assert (> (count (curry = m) xs) (quotient n 2))) #:guarantee (assert (= m (boyer-moore xs))))
  location...:
   eval:6:2
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/form/module.rkt:33:2: try-next
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [34]:
; Check the algorithm for a list of n arbitrary integers.
(define (check2 n [bw #f])
  (current-bitwidth bw)
  (define-symbolic* xs integer? [n])
  (define-symbolic* m integer?)
  (verify
   (when ; there is a majority value ...
       (> (count (curry = m) xs)
          (quotient n 2))
     (assert (= m (boyer-moore xs)))))) ; algorithm works

(time (check2 1)) 


application: not a procedure;
 expected a procedure that can be applied to arguments
  given: 1
  context...:
   body of top-level
   eval:2:0: check2


In [35]:
(clear-vc!)


#### 3. Sudoku

In [57]:
#lang iracket/lang #:require rosette

In [59]:
(define (choice type) 
  (define-symbolic* c type) 
  c)

(define (more-or-less-1 x)
  (if (choice boolean?)
      (+ x 1)
      (- x 1)))

(solve (assert (= 0 (more-or-less-1 1)))) ; #f
(solve (assert (= 2 (more-or-less-1 1)))) ; #t

(model
 [a #t]
 [b #t]
 [c$6 #t])

In [60]:
(assert #f)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level


In [61]:
(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x 1) (+ x 2) (+ x 2)))
 
(define (same p f x)
 (assert (= (p x) (f x))))

In [66]:
(same poly factored 0)

In [63]:
(same poly factored -1)

In [64]:
(same poly factored -2)

In [2]:
#lang iracket/lang #:require rosette/safe 

(define-symbolic l h int32?)

(define cex (verify (check-mid bvmid l h)))


int32?: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [6]:
#lang iracket/lang #:require yosys
; SMT-LIBv2 description generated by Yosys 0.23+25 (git sha1 13e4f343b, clang 10.0.0-4ubuntu1 -fPIC -Os)
; yosys-smt2-stdt
; yosys-smt2-module adder
(declare-datatype |adder_s| ((|adder_mk|
  (|adder_is| Bool)
  (|adder#0| (_ BitVec 32)) ; \a
  (|adder#1| (_ BitVec 32)) ; \b
)))
; yosys-smt2-input a 32
; yosys-smt2-witness {"offset": 0, "path": ["\\a"], "smtname": "a", "smtoffset": 0, "type": "input", "width": 32}
(define-fun |adder_n a| ((state |adder_s|)) (_ BitVec 32) (|adder#0| state))
; yosys-smt2-input b 32
; yosys-smt2-witness {"offset": 0, "path": ["\\b"], "smtname": "b", "smtoffset": 0, "type": "input", "width": 32}
(define-fun |adder_n b| ((state |adder_s|)) (_ BitVec 32) (|adder#1| state))
(define-fun |adder#2| ((state |adder_s|)) (_ BitVec 32) (bvadd (|adder#0| state) (|adder#1| state))) ; \c
; yosys-smt2-output c 32
(define-fun |adder_n c| ((state |adder_s|)) (_ BitVec 32) (|adder#2| state))
(define-fun |adder_a| ((state |adder_s|)) Bool true)
(define-fun |adder_u| ((state |adder_s|)) Bool true)
(define-fun |adder_i| ((state |adder_s|)) Bool true)
(define-fun |adder_h| ((state |adder_s|)) Bool true)
(define-fun |adder_t| ((state |adder_s|) (next_state |adder_s|)) Bool true) ; end of module adder
; yosys-smt2-topmod adder
; end of yosys output


#%top-interaction: unbound identifier;
 also, no #%app syntax transformer is bound
  at: #%top-interaction
  in: (#%top-interaction declare-datatype adder_s ((adder_mk (adder_is Bool) (adder#0 (_ BitVec 32)) (adder#1 (_ BitVec 32)))))
  context...:
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [63]:
(add_n 1)

#%top-interaction: unbound identifier;
 also, no #%app syntax transformer is bound
  at: #%top-interaction
  in: (#%top-interaction add_n 1)


In [21]:
(clear-vc!)

In [4]:
(define (adder_spec a b)
  (+ a b))

In [1]:
(adder_spec 10 15)

adder_spec: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   body of top-level


In [7]:
(require "a.rkt")
(f 3)

Hello4
